In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [6]:

pd.read_sql(
    '''
    SELECT CustomerName, Country, Address, City
    FROM Customers
    WHERE Country in ("Germany","France","Madrid") OR city = "Madrid"
    ''',
    con,
)

,CustomerName,Country,Address,City
0,Alfreds Futterkiste,Germany,Obere Str. 57,Berlin
1,Blauer See Delikatessen,Germany,Forsterstr. 57,Mannheim
2,Blondel père et fils,France,"24, place Kléber",Strasbourg
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67",Madrid
4,Bon app',France,"12, rue des Bouchers",Marseille
5,Drachenblut Delikatessend,Germany,Walserweg 21,Aachen
6,Du monde entier,France,"67, rue des Cinquante Otages",Nantes
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86",Madrid
8,Folies gourmandes,France,"184, chaussée de Tournai",Lille
9,Frankenversand,Germany,Berliner Platz 43,München


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [7]:
pd.read_sql(
    '''
    SELECT Country, Count(Country) as Count 
    FROM Customers 
    GROUP BY Country
    ORDER BY Count(*) DESC
    ''',
    con,
)

,Country,Count
0,USA,13
1,Germany,11
2,France,11
3,Brazil,9
4,UK,7
5,Spain,5
6,Mexico,5
7,Venezuela,4
8,Italy,3
9,Canada,3


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [8]:
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate
    FROM Shippers 
    left join Orders
    on Orders.ShipperID = Shippers.ShipperID
    ORDER BY OrderID ASC
    LIMIT 1 OFFSET 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [9]:
pd.read_sql(
    '''
    SELECT ProductName, Price
    FROM Products
    join OrderDetails
    on Products.ProductID = OrderDetails.ProductID
    where OrderID = 
        (
            select a.OrderID as OrderID
            from
            (
            SELECT OrderID, sum(Price) as Price 
            FROM Products
            join OrderDetails
            on Products.ProductID = OrderDetails.ProductID 
            Group by OrderID
            Order by Price Desc
            limit 1 ) as a
        )
    ''',
    con,
)


,ProductName,Price
0,Rössle Sauerkraut,45.60
1,Thüringer Rostbratwurst,123.79
2,Côte de Blaye,263.50
3,Maxilaku,20.00
4,Tourtière,7.45


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [10]:
pd.read_sql(
    '''
    SELECT OrderID, ProductName, Quantity
    FROM Products
    join OrderDetails
    on Products.ProductID = OrderDetails.ProductID
    where ProductName in
            (
                select a.ProductName as ProductName
                from
                    (SELECT ProductName,sum(Quantity) as Quantity
                    FROM Products
                    join OrderDetails
                    on Products.ProductID = OrderDetails.ProductID
                    group by ProductName
                    order by Quantity desc
                    limit 1) a
            )
    group by OrderID
    order by Quantity desc
    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10359,Gorgonzola Telino,70
1,10390,Gorgonzola Telino,60
2,10342,Gorgonzola Telino,56
3,10376,Gorgonzola Telino,42
4,10272,Gorgonzola Telino,40
5,10393,Gorgonzola Telino,32
6,10374,Gorgonzola Telino,30
7,10356,Gorgonzola Telino,30
8,10335,Gorgonzola Telino,25
9,10363,Gorgonzola Telino,20


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [11]:
pd.read_sql(
    '''
    SELECT SupplierName, Country, ContactName, Phone
    FROM Suppliers
    where SupplierID in 
        (
            select a.SupplierID
            from
            (SELECT SupplierID, count(OrderID) as Sum_Orders
            FROM Products
            JOIN OrderDetails
            on Products.ProductID = OrderDetails.ProductID
            group by SupplierID
            order by Sum_Orders desc
            limit 5) as a
        )
    ''', 
    con, 
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [12]:
pd.read_sql(
    '''
    SELECT  Country, CategoryName, sum(Price) as 'sum_by_orders'

    FROM Products
    join OrderDetails
        on Products.ProductID = OrderDetails.ProductID
    join Categories
        on Products.CategoryID = Categories.CategoryID
    join Orders
        on Orders.OrderID = OrderDetails.OrderID
    join Customers
        on Customers.CustomerID = Orders.CustomerID
    
    where Country = 'Brazil'
    group by Country, CategoryName
    order by sum_by_orders desc
    limit 1
    ''',
    con,
)

,Country,CategoryName,sum_by_orders
0,Brazil,Beverages,405.75


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [13]:
pd.read_sql(
    '''
    
        SELECT *
        FROM 
        (select Country, c.OrderID as OrderID, sum(Price) as 'sum_order_price'
        from Customers a
        join Orders b
            on a.CustomerID = b.CustomerID 
        join OrderDetails c 
            on b.OrderID = c.OrderID 
        join Products d
            on c.ProductID = d.ProductID 
            
            where Country = 'USA'
            group by c.OrderID
            order by sum_order_price desc
            limit 1) as e
    
    union all
    
        SELECT *
        FROM 
        (select Country, c.OrderID as OrderID, sum(Price) as 'sum_order_price'
        from Customers a
        join Orders b
            on a.CustomerID = b.CustomerID 
        join OrderDetails c 
            on b.OrderID = c.OrderID 
        join Products d
            on c.ProductID = d.ProductID 
            
            where Country = 'USA'
            group by c.OrderID
            order by sum_order_price asc
            limit 1) as e
    
    ''',
    con,
)

,Country,OrderID,sum_order_price
0,USA,10329,336.59
1,USA,10271,2.50


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [14]:
pd.read_sql(
    '''
    select sum_orders, (FirstName || " " ||  LastName) as 'Full_Name'
    from
    (SELECT LastName, FirstName, Photo as Birth_info , BirthDate, count(OrderID) as 'sum_orders'
    FROM Employees a
    join Orders b
    on a.EmployeeID = b.EmployeeID
    group by FirstName
    order by Photo desc
    limit 4
    ) as a
    ''',
    con,
)

,sum_orders,Full_Name
0,6,Anne Dodsworth
1,27,Laura Callahan
2,14,Robert King
3,18,Michael Suyama


10. Сколько банок крабового мяса всего было заказано.

In [15]:
pd.read_sql(
    '''
    SELECT ProductName, sum(Quantity) as Quantity
    FROM OrderDetails a
    join Products b
    on a.ProductID = b.ProductID
    where ProductName like '%Crab%'
    ''',
    con,
)

,ProductName,Quantity
0,Boston Crab Meat,256
